In [ ]:
#CONEXÃO GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#INSTALANDO BIBLIOTECAS
!pip install pandas

In [ ]:
def carregar_dfs(cargo,pasta_base, ano):
  df1 = pd.read_csv(f'{pasta_base}/{cargo}XMESADMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)#Mes admissão
  df2 = pd.read_csv(f'{pasta_base}/{cargo}XMESDESX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)#Mês demissão
  df3 = pd.read_csv(f'{pasta_base}/{cargo}XFTEX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)#Faixa Tempo de Emprego
  df4 = pd.read_csv(f'{pasta_base}/{cargo}XFRMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)#Faixa Remuneração Media
  return [df1, df2, df3, df4]

In [ ]:
def renomear_dfs(lista_dfs):
  for i in range(len(lista_dfs)):
      df = lista_dfs[i]
      lista_dfs[i].rename(columns={'Unnamed: 0':'Municipio',
                                   'Unnamed: 1':'Faixa Etaria'},inplace=True)
  return lista_dfs

In [ ]:
def remove_linha(lista_dfs):
    for i in range(len(lista_dfs)):
        lista_dfs[i] = lista_dfs[i].dropna(subset=['Faixa Etaria']).reset_index(drop=True)
        lista_dfs[i] = lista_dfs[i][~lista_dfs[i]['Faixa Etaria'].str.strip().isin(['Critério', 'igual a'])].reset_index(drop=True)
    return lista_dfs


In [29]:
def Completar_Municipio(lista_dfs):
   for x in range(len(lista_dfs)):
              dfs[x]['Municipio'] = dfs[x]['Municipio'].ffill()
              dfs[x]['Municipio'] = dfs[x]['Municipio'].replace('{ñ class}', 'N/I')
   return lista_dfs

In [ ]:
def Transpor_valor(lista_dfs):
  for i in range(len(lista_dfs)):
    id_vars = ['Municipio', 'Faixa Etaria']
    value_vars = [col for col in lista_dfs[i].columns if col not in id_vars]
    if i== 0:
        var_name = 'Periodo'
    if i== 1:
        var_name = 'Periodo'
    if i== 2:
        var_name = 'Tempo_Emprego'
    if i== 3:
        var_name = 'Media_Remu'
    df_Temp =lista_dfs[i].melt(
       id_vars=id_vars,
       value_vars=value_vars,
       var_name=var_name,
       value_name='Total')
    df_Temp['Total'] = df_Temp['Total'].astype(int)
    lista_dfs[i] = df_Temp
  return lista_dfs


In [ ]:
def Sexo_Func(lista_dfs):
  for i in range(len(lista_dfs)):
      lista_dfs[i]["Sexo"] = lista_dfs[i].iloc[:, 2].astype(str).apply(lambda x: "Feminino" if x.strip().endswith(".1") else "Masculino")
  return lista_dfs

In [ ]:
def Linha_A_Linha(lista_dfs):
  for i in range(len(lista_dfs)):
      # Separar as linhas por categorias
      linhas_zero = lista_dfs[i][lista_dfs[i]['Total'] == 0]
      linhas_um = lista_dfs[i][lista_dfs[i]['Total'] == 1]
      linhas_maior_que_um = lista_dfs[i][lista_dfs[i]['Total'] > 1]
      # Repetir as linhas com ValorFaixa > e ajustar
      linhas_repetidas = linhas_maior_que_um.loc[
          linhas_maior_que_um.index.repeat(linhas_maior_que_um['Total'])
      ].copy()
      linhas_repetidas['Total'] = 1
      # concatenando
      df_final = pd.concat([ linhas_um, linhas_repetidas, linhas_zero],ignore_index=True)
      # Ordenando e resetando o índice
      df_final = df_final.sort_values(by=lista_dfs[i].columns.tolist()).reset_index(drop=True)
      lista_dfs[i] = df_final
  return lista_dfs

In [ ]:
def Status(lista_dfs):
    status_map = {0: 'Admitido', 1: 'Demitido', 2: 'Ativo', 3: 'Ativo'}
    for i, df in enumerate(lista_dfs):
        df['Status'] = status_map.get(i, 'Desconhecido')
    return lista_dfs

In [ ]:
def Removendo_0_ativos(lista_dfs):
  dfs[2] = dfs[2][dfs[2]['Total'] != 0]
  dfs[3] = dfs[3][dfs[3]['Total'] != 0]
  return lista_dfs

In [ ]:
def Manipulando_Ativos(lista_dfs,ano):
  lista_dfs = Removendo_0_ativos(lista_dfs)
  df_fte_f = lista_dfs[2][lista_dfs[2]['Sexo'] == 'Feminino']
  df_frm_f = lista_dfs[3][lista_dfs[3]['Sexo'] == 'Feminino']
  df_fte_m = lista_dfs[2][lista_dfs[2]['Sexo'] == 'Masculino']
  df_frm_m = lista_dfs[3][lista_dfs[3]['Sexo'] == 'Masculino']
  df_m = pd.concat([df_fte_m[['Municipio', 'Faixa Etaria','Sexo','Status','Total',
  'Tempo_Emprego']].reset_index(drop=True), df_frm_m[['Media_Remu']].reset_index(drop=True)], axis=1)
  df_f = pd.concat([df_fte_f[['Municipio', 'Faixa Etaria','Sexo','Status','Total',
  'Tempo_Emprego']].reset_index(drop=True), df_frm_f[['Media_Remu']].reset_index(drop=True)], axis=1)
  df_novo = pd.concat([df_m, df_f], axis=0, ignore_index=True)
  df_novo['Periodo'] = ano
  return df_novo

In [ ]:
def Criando_Colunas(lista_dfs):
     for i in range(len(lista_dfs)):
        if i < 2 :
            lista_dfs[i]['Tempo_Emprego'] = '-'
            lista_dfs[i]['Media_Remu'] = '-'
            lista_dfs[i]['Periodo'] = lista_dfs[i]['Periodo'].str.replace(r'\.1$', '', regex=True)
     return lista_dfs

In [ ]:
def Concatenando_dfs(lista_dfs,df):
  lista_dfs = Criando_Colunas(lista_dfs)
  df_final = pd.concat([lista_dfs[0],lista_dfs[1],df], axis=0, ignore_index=True)
  df_final['Tempo_Emprego'] = df_final['Tempo_Emprego'].str.replace(r'\.1$', '', regex=True)
  df_final['Media_Remu'] = df_final['Media_Remu'].str.replace(r'\.1$', '', regex=True)
  return df_final

In [ ]:
def Add_uf_estado(df):
  #sigla estados
  df['sigla-uf'] = df['Municipio'].apply(lambda x: 'N/I' if x == 'N/I' else str(x)[:2])
  #nome estados
  estados = {
      'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
      'BA': 'Bahia', 'CE': 'Ceará', 'ES': 'Espírito Santo', 'GO': 'Goiás',
      'MA': 'Maranhão', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
      'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba', 'PR': 'Paraná',
      'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro',
      'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul', 'RO': 'Rondônia',
      'RR': 'Roraima', 'SC': 'Santa Catarina', 'SP': 'São Paulo',
      'SE': 'Sergipe', 'TO': 'Tocantins','DF': 'Distrito Federal','N/I':'N/I'
  }
  df['UF'] = df['sigla-uf'].map(estados)
  #regiao
  regiao = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte','RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste','PB': 'Nordeste', 'PE': 'Nordeste',
    'PI': 'Nordeste',    'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste','MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul',
    'N/I': 'N/I'  }
  df['Regiao'] = df['sigla-uf'].map(regiao)
  #tratamento nomes municipios
  df['Municipio'] = df['Municipio'].apply(lambda x: x if x == 'N/I' else str(x)[3:])
  return df

In [ ]:
def Cargo_Func(df,cargo): # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
    if cargo == 'ANALISTA':
     cargo='ANALISTA DE SISTEMAS COMPUTACIONAIS'
     df["Familia CBO 2002"] = cargo
    if cargo == 'DIRETORES':
     cargo='DIRETORES DE SERVICOS DE INFORMATICA'
     df["Familia CBO 2002"] = cargo
    if cargo == 'ENG':
     cargo='ENGENHEIROS EM COMPUTACAO'
     df["Familia CBO 2002"] = cargo
    if cargo == 'ESP':
     cargo='ESPECIALISTAS EM INFORMATICA'
     df["Familia CBO 2002"] = cargo
    if cargo == 'TECP':
     cargo='TECNICOS EM PROGRAMACAO'
     df["Familia CBO 2002"] = cargo
    if cargo == 'TECO':
     cargo='TECNICOS EM OPERACAO E MONITORAMENTO DE COMPUTADORES'
     df["Familia CBO 2002"] = cargo
    if cargo == 'GER':
     cargo='GERENTES DE TECNOLOGIA DA INFORMACAO'
     df["Familia CBO 2002"] = cargo
    return df


In [ ]:
def Criando_ano(df,ano):
  df['Ano'] = ano
  return df

In [ ]:
def Ordenar_colunas(df):
  ordem_colunas = ['Ano','Regiao','UF','sigla-uf','Municipio', 'Familia CBO 2002','Sexo',
                   'Faixa Etaria','Periodo','Status','Tempo_Emprego', 'Media_Remu','Total']
  df = df[ordem_colunas]
  df = df.sort_values(by=['Regiao','UF','sigla-uf','Municipio'])
  return df

In [ ]:
import pandas as pd
import numpy as np

ano = 2015
while ano <= 2024:
 familia = ['ANALISTA','DIRETORES','ENG','ESP','TECP','TECO','GER'] # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
 pasta_raiz = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Familias_CBO_Dados' #ALTERAR PASTA
 pasta_anos = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Familia_CBO_Anos' #ALTERAR PASTA
 pasta_base = f'{pasta_raiz}/{ano}'
 for i in range(len(familia)):
      dfs=[]
      cargo = familia[i]
      dfs = carregar_dfs(cargo,pasta_base, ano)
      df1, df2, df3, df4 = dfs
      dfs = [df1, df2, df3, df4]
      dfs = renomear_dfs(dfs)
      dfs = remove_linha(dfs)
      dfs = Completar_Municipio(dfs)
      dfs = Transpor_valor(dfs)
      dfs = Sexo_Func(dfs)
      dfs = Linha_A_Linha(dfs)
      dfs = Status(dfs)
      df = Manipulando_Ativos(dfs,ano)
      df6 = Concatenando_dfs(dfs,df)
      df6 = Add_uf_estado(df6)
      df6 = Criando_ano(df6,ano)
      df6 = Cargo_Func(df6,cargo)
      df6 = Ordenar_colunas(df6)
      df6.to_csv(f'{pasta_anos}/{cargo}X{ano}.csv',index=False,sep=';', encoding='utf-8')
      print(f'Arquivo {cargo}X{ano}.csv gerado com sucesso!')
      del df1, df2, df3, df4, df, df6
 ano += 1


Arquivo ANALISTAX2015.csv gerado com sucesso!
Arquivo DIRETORESX2015.csv gerado com sucesso!
Arquivo ENGX2015.csv gerado com sucesso!
Arquivo ESPX2015.csv gerado com sucesso!
Arquivo TECPX2015.csv gerado com sucesso!
Arquivo TECOX2015.csv gerado com sucesso!
Arquivo GERX2015.csv gerado com sucesso!
Arquivo ANALISTAX2016.csv gerado com sucesso!
Arquivo DIRETORESX2016.csv gerado com sucesso!
Arquivo ENGX2016.csv gerado com sucesso!
Arquivo ESPX2016.csv gerado com sucesso!
Arquivo TECPX2016.csv gerado com sucesso!
Arquivo TECOX2016.csv gerado com sucesso!
Arquivo GERX2016.csv gerado com sucesso!
Arquivo ANALISTAX2017.csv gerado com sucesso!
Arquivo DIRETORESX2017.csv gerado com sucesso!
Arquivo ENGX2017.csv gerado com sucesso!
Arquivo ESPX2017.csv gerado com sucesso!
Arquivo TECPX2017.csv gerado com sucesso!
Arquivo TECOX2017.csv gerado com sucesso!
Arquivo GERX2017.csv gerado com sucesso!
Arquivo ANALISTAX2018.csv gerado com sucesso!
Arquivo DIRETORESX2018.csv gerado com sucesso!
Arquivo

In [ ]:
import pandas as pd
import numpy as np

ano = 2015
while ano <= 2024:
    print(f'Ano: {ano}')
    familia = ['ANALISTA','DIRETORES','ENG','ESP','TECP','TECO','GER'] # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
    pasta_raiz = f'/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/' #ALTERAR PASTA
    pasta_base = f'{pasta_raiz}Familia_CBO_Anos/'
    for i in range(len(familia)):
        cargo = familia[i]
        df_temp = pd.read_csv(f'{pasta_base}/{cargo}X{ano}.csv',sep=';', encoding='utf-8',low_memory=False)
        if i == 0:
            df_geral = pd.DataFrame()
        df_geral = pd.concat([df_geral, df_temp], ignore_index=True)
        print(f'Arquivo {cargo}X{ano}.csv carregado com sucesso!')
    df_geral.to_csv(f'{pasta_raiz}/Familias_CBO_Manipulados/GeralX{ano}.csv',index=False,sep=';', encoding='utf-8')
    del df_geral, df_temp
    ano += 1

Ano: 2015
Arquivo ANALISTAX2015.csv carregado com sucesso!
Arquivo DIRETORESX2015.csv carregado com sucesso!
Arquivo ENGX2015.csv carregado com sucesso!
Arquivo ESPX2015.csv carregado com sucesso!
Arquivo TECPX2015.csv carregado com sucesso!
Arquivo TECOX2015.csv carregado com sucesso!
Arquivo GERX2015.csv carregado com sucesso!
Ano: 2016
Arquivo ANALISTAX2016.csv carregado com sucesso!
Arquivo DIRETORESX2016.csv carregado com sucesso!
Arquivo ENGX2016.csv carregado com sucesso!
Arquivo ESPX2016.csv carregado com sucesso!
Arquivo TECPX2016.csv carregado com sucesso!
Arquivo TECOX2016.csv carregado com sucesso!
Arquivo GERX2016.csv carregado com sucesso!
Ano: 2017
Arquivo ANALISTAX2017.csv carregado com sucesso!
Arquivo DIRETORESX2017.csv carregado com sucesso!
Arquivo ENGX2017.csv carregado com sucesso!
Arquivo ESPX2017.csv carregado com sucesso!
Arquivo TECPX2017.csv carregado com sucesso!
Arquivo TECOX2017.csv carregado com sucesso!
Arquivo GERX2017.csv carregado com sucesso!
Ano: 20